# Face Detection Evaluation and Experimentation Notebook
Notebook used for experiments and evaluations on the face detection algorithm used in this project.  
Currently finds the accuracy and IoU values for every face and frame

In [ ]:
# Sometimes an error where 
%pip install seaborn

In [ ]:
import cv2, torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

from dataLoader import Train_Loader, Val_Loader
from utils import tools
from faceDetection.faceDetector import FaceDetection
from evaluation import face_evaluate, general_face_evaluation

### Intersection over Union
Code to calculate Intersection over Union values for the faces detected.  
Somewhat misleading since we have to resize the frames for face detection to work and bounding boxes for the faces in the annotations/labels are for the original frame before resizing.  
We aren't expecting good results

In [ ]:
def get_IoU(prediction, actual):
    '''
    Function used to get IoU values for faces detected

    Params:
        prediction: faces detected using our algorithm
        actual: annotation/label for the frame from the csv file

    Return: Array of IoU values for each face detected
    '''
    
    ious = []
    # Get face bound boxes for the labels
    if torch.is_tensor(actual[1]):
        a_faces = actual[1].numpy()
    else:
        a_faces = actual[1]

    # If no label is returned for the frame 
    if len(prediction) == 0:
        return [0]

    # Evaluates if there is more than one label for the frame
    if len(a_faces.shape) > 1:
        for i in range(len(a_faces)):
            for j in range(len(prediction)):
                # Checks if bounding box for face detected is correct
                # Then compares the predicted label with the actual label and returns the counts
                bound_box = prediction[j][3:7]
                if face_evaluate(bound_box, a_faces[i]):
                    iou = calculate_IoU(bound_box, a_faces[i])
                    ious.append(iou)

    # Evaluation for frames with single labels   
    else:
        for j in range(len(prediction)):
            bound_box = prediction[j][3:7]
            if face_evaluate(bound_box, a_faces):
                iou = calculate_IoU(bound_box, a_faces)
                ious.append(iou)

    return ious

In [ ]:
def calculate_IoU(predicted, actual):
    '''
    Function used to calculate the intersection over union (IoU) values for two bounding boxes
    Ammended from credit: https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/

    Params:
        predicted: detected bounding box using our algorithm
        actual: Actual bound box found in the labels/annotation
    
    Return: float IoU value
    '''
    
    x1, y1, x2, y2 = predicted * 300
    a_x1, a_y1, a_x2, a_y2 = actual * 300

    # Get area of each bounding box
    # +1 used to avoid 0 values
    predicted_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    actual_area = (a_x2 - a_x1 + 1) * (a_y2 - a_y1 + 1)

    # Get coordinates for the intersection box
    m_x1 = max(x1, a_x1)
    m_x2 = min(x2, a_x2)

    m_y1 = max(y1, a_y1)
    m_y2 = min(y2, a_y2)

    # Calculate intersection and union areas to get IoU
    inter = max(0, (m_x2 - m_x1 + 1)) * max(0, (m_y2 - m_y1 + 1))
    union = float(predicted_area + actual_area - inter)
    
    if union == 0:
        return 0

    iou = inter / union
    return iou

### Evaluation Code
Code to run the evaluations  
Some functions used can be found in the evaluation.py file   
Can change videos used by editing the first two lines of the file  
Can also change the threshold value for the face detection algorithm which is the probabilty whether a face is detected in the return area  

In [ ]:
# ids = ['_mAfwH6i90E', 'B1MAUxpKaV8', '7nHkh4sP5Ks', '2PpxiG0WU18', '-5KQ66BBWC4', '5YPjcdLbs5g', '20TAGRElvfE', '2fwni_Kjf2M']
ids = ['B1MAUxpKaV8']
threshold = 0.5

correct = 0
total = 0
faces_detected = 0
ious = []

for video_id in ids:
    trainLoader = Train_Loader(video_id=video_id, root_dir=video_id)
    trainLoaded = DataLoader(trainLoader, batch_size=64, num_workers=0, shuffle=False)

    for images, labels in trainLoaded:
        for i in range(len(images)):
            actual_label = trainLoader.extract_labels(trainLoader.labels, labels, i)

            # Apply Face Detection Algorithm
            face_detect = FaceDetection(images[i].numpy(), threshold)
            faces = face_detect.detect()
            # Get evaluation values -uses function from evaluation file
            vid_correct, vid_total = general_face_evaluation(faces, actual_label)
            ious.append(get_IoU(faces, actual_label))

            correct += vid_correct
            total += vid_total
            faces_detected += len(faces)

    # print(f'------ {ids} ------')
    # print(f'Correct: {vid_correct}')
    # print(f'Total: {vid_total}')
    # print(f'Accuracy: {vid_correct/vid_total}')


accuracy = correct / total
# Converts md array to 1d
ious = list(np.concatenate(ious).flat)

print(f'------ Total Face Evaluation ------')
print(f'Correct: {correct}')
print(f'Total Number of Faces in labels: {total}')
print(f'Total Number of Faces Detected:  {faces_detected}')
print(f'Accuracy: {accuracy}')

print(f'\n------------IoU----------------')     
print(f'Mean: {np.mean(ious)}')
print(f'Median: {np.median(ious)}')       
print(f'Max: {max(ious)}')
print(f'Min: {min(ious)}')
